In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('recomended').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator 

In [7]:
df = spark.read.csv('/home/aravind/Downloads/movielens_ratings.csv',inferSchema=True,header=True)

In [8]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
Train_data, Test_data = df.randomSplit([0.8,0.2])

In [11]:
Train_data.describe().show()

+-------+-----------------+------------------+------------------+
|summary|          movieId|            rating|            userId|
+-------+-----------------+------------------+------------------+
|  count|             1204|              1204|              1204|
|   mean|49.07475083056478|1.7350498338870433|14.361295681063122|
| stddev|29.06322503435479| 1.157030947141589| 8.645884494114373|
|    min|                0|               1.0|                 0|
|    max|               99|               5.0|                29|
+-------+-----------------+------------------+------------------+



In [12]:
Test_data.describe().show()

+-------+-----------------+------------------+------------------+
|summary|          movieId|            rating|            userId|
+-------+-----------------+------------------+------------------+
|  count|              297|               297|               297|
|   mean|50.74747474747475|1.9326599326599327|14.474747474747474|
| stddev|28.42855320241671|1.2927202601977241| 8.378717530320941|
|    min|                0|               1.0|                 0|
|    max|               99|               5.0|                29|
+-------+-----------------+------------------+------------------+



In [13]:
als =ALS(maxIter=5,regParam=0.01,userCol='userId',itemCol='movieId',ratingCol='rating')

In [14]:
model = als.fit(Train_data)

In [15]:
prediction = model.transform(Test_data)

In [16]:
prediction.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|     31|   1.0|     5| -0.25254714|
|     31|   1.0|     0|   2.2742794|
|     85|   1.0|    12|   0.5628958|
|     85|   2.0|    20|   1.7187313|
|     65|   1.0|    22|   3.8931248|
|     53|   3.0|    13| -0.30045214|
|     53|   1.0|     6|    2.338067|
|     53|   2.0|    19|   1.9799979|
|     53|   1.0|    23|    1.978471|
|     53|   1.0|     7|   2.5822096|
|     53|   5.0|    21|   1.2093623|
|     34|   1.0|     4|   2.9059067|
|     81|   1.0|    16|   1.0299227|
|     81|   2.0|     9|   2.2874467|
|     28|   1.0|     6|  0.61559165|
|     28|   1.0|    14|  -0.9485489|
|     28|   5.0|    18|-0.081665084|
|     76|   3.0|     3| -0.21729118|
|     76|   3.0|     7|  0.74417406|
|     76|   5.0|    14|-0.011273269|
+-------+------+------+------------+
only showing top 20 rows



In [23]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [24]:
rmse = evaluator.evaluate(prediction)

In [25]:
print("RMSE Value")
print(rmse)

RMSE Value
2.137412570382095


In [26]:
single_user = Test_data.filter(Test_data['userId']==11).select(['movieId','userId'])

In [27]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     27|    11|
|     35|    11|
|     36|    11|
|     40|    11|
|     43|    11|
|     47|    11|
|     67|    11|
|     70|    11|
|     75|    11|
|     86|    11|
+-------+------+



In [28]:
recomendation = model.transform(single_user)

In [29]:
recomendation.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     11|    11| 3.4236126|
|     27|    11| 2.2846847|
|     40|    11| 2.2516387|
|     36|    11|   1.97287|
|     70|    11| 1.5440751|
|     86|    11| 1.3056488|
|     67|    11| 1.0690862|
|     35|    11|0.53086364|
|     75|    11|0.40965062|
|     43|    11| 0.2216385|
|     47|    11|-0.9032193|
+-------+------+----------+

